In [ ]:
import panel as pn
import jsonschema
import json
from panel.widgets import TextInput, Button, Select, IntInput
from IPython.display import display, clear_output
from library.utils.widgets import MessageBox
import traceback

margin = (10, 10, 5, 10)

class Metadata:

    def __init__(self, schema):

        self.schema = schema

        pn.extension()
        self._form_box = pn.WidgetBox()
        self._msg_output = MessageBox()

        self.create_widgets(schema)

        display(self._form_box)
        display(self._msg_output)


    def create_widgets(self, schema):

        for key, value in schema["properties"].items():
            self._form_box.append(self.create_input_widget(value))


    def create_input_widget(self, schema):
        if "enum" in schema:
            return Select(name=schema["title"], options=schema["enum"], margin=margin)
        elif schema["type"] == "array":
            return self.genetate_array_widget(schema)
        elif schema["type"] == "string":
            return TextInput(name=schema["title"], margin=margin)
        elif schema["type"] == "number":
            return IntInput(name=schema["title"], margin=margin)


    def genetate_array_widget(self, schema):
        box =  pn.WidgetBox(margin=margin)
        title = schema["title"]
        box.append(pn.pane.Markdown(title))

        column = pn.Column(margin=margin)

        def create_items():
            column_list = column.objects
            title_num = title + str(len(column_list) + 1)
            items = schema["items"]
            if items["type"] == "object":
                obj_box = pn.WidgetBox()
                obj_box.append(pn.pane.Markdown(title_num))
                for key, value in items["properties"].items():
                    obj_box.append(self.create_input_widget(value))
                return pn.Row(obj_box, create_remove_button(obj_box))
            else:
                widget = self.create_input_widget({
                    "type": items["type"],
                    "title": title_num
                })
                return pn.Row(widget, create_remove_button(widget))

        def create_remove_button(widget):
            remove_button = Button(name='Remove', button_type='danger', button_style='outline')
            def remove_item(event):
                try:
                    # 指定されたwidgetを削除
                    objects = column.objects
                    index = 0
                    for i, obj in enumerate(objects):
                        now = obj[0]
                        if now == widget:
                            index = i
                            break
                    objects.pop(index)
                    # 表示を更新
                    column.clear()
                    for i, obj in enumerate(objects):
                        title_num = title + str(i + 1)
                        w = obj[0]
                        if isinstance(w, pn.WidgetBox):
                            wb_list = w.objects
                            wb_list[0].object = title_num
                            w.clear()
                            w.extend(wb_list)
                        else:
                            w.name = title_num
                        column.append(obj)
                except Exception as e:
                    message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
                    self._msg_output.update_error(message)
            remove_button.on_click(remove_item)
            return remove_button

        def add_item(event=None):
            column.append(create_items())

        add_item()
        box.append(column)

        add_button = Button(name='Add', button_type='primary', button_style='outline')
        add_button.on_click(add_item)
        box.append(add_button)

        return box